# CMIP6 Quantile

In [ ]:
%%capture
!pip install netcdf4 metpy colorcet
import os, fnmatch, cv2, gzip, shutil, datetime, numpy as np, pandas as pd, xarray as xr
import pylab as plt, plotly.express as px, seaborn as sns, colorcet as cc, altair as alt
from matplotlib.gridspec import GridSpec; from matplotlib.colors import from_levels_and_colors as flc
from metpy.units import units; import metpy.calc as mpcalc

!pip install --upgrade zarr gcsfs cftime nc-time-axis
import fsspec, zarr, gcsfs; gcs = gcsfs.GCSFileSystem( token = 'anon' )

%load_ext google.colab.data_table 
from google.colab import drive; drive.mount( '/content/drive' )

## Get list of models

In [ ]:
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv' )
df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp585' & table_id == 'day'" )
tas = df1.source_id.unique().tolist()
df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == 'ssp585' & table_id == 'day'" )
huss = df1.source_id.unique().tolist()
ssp585_models = list( set(tas).intersection( set(huss) ) )

df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp126' & table_id == 'day'" )
tas = df1.source_id.unique().tolist()
df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == 'ssp126' & table_id == 'day'" )
huss = df1.source_id.unique().tolist()
ssp126_models = list( set(tas).intersection( set(huss) ) )

df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp245' & table_id == 'day'" )
tas = df1.source_id.unique().tolist()
df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == 'ssp245' & table_id == 'day'" )
huss = df1.source_id.unique().tolist()
ssp245_models = list( set(tas).intersection( set(huss) ) )

df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp370' & table_id == 'day'" )
tas = df1.source_id.unique().tolist()
df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == 'ssp370' & table_id == 'day'" )
huss = df1.source_id.unique().tolist()
ssp370_models = list( set(tas).intersection( set(huss) ) )

df1 = df.query( "activity_id=='CMIP' & variable_id == 'tas' & experiment_id == 'historical' & table_id == 'day'" )
tas = df1.source_id.unique().tolist()
df1 = df.query( "activity_id=='CMIP' & variable_id == 'huss' & experiment_id == 'historical' & table_id == 'day'" )
huss = df1.source_id.unique().tolist()
hist_models = list( set(tas).intersection( set(huss) ) )
hist_models

## loop through all models for historical
RH and dewpoint from the method of Bolton 1980 (http://www.eol.ucar.edu/projects/ceop/dm/documents/refdata_report/eqns.html)


In [ ]:
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv' )
 
#bad time 'SAM0-UNICON'

for model in hist_models:
    print(model)
    df1 = df.query( f"activity_id=='CMIP' & variable_id == 'huss' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
    huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
    huss = huss.sel(time = slice('1985-01-01T12:00:00', '2014-12-30T12:00:00'))

    df1 = df.query( f"activity_id=='CMIP' & variable_id == 'tas' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
    Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
    Ta = Ta.sel(time = slice('1985-01-01T12:00:00', '2014-12-30T12:00:00')) - 273.15

    RH = (huss.huss * 1013 / (0.378 * huss.huss + 0.622)) / (6.112 * np.exp((17.67 * Ta.tas)/(Ta.tas + 243.5)))
    RH = xr.where(RH < 1, RH, 1)

    thi = (1.8 * Ta.tas + 32) - ( (0.55 - 0.55 * RH) * (1.8 * Ta.tas - 26) )
    thi = thi.chunk({'time': -1})
    thi = thi.groupby('time.dayofyear').median('time').to_dataset( name = 'THI' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/historical/{model}_hist_1985_2014_thi_doy_median.nc')


In [ ]:
def run_ssp(ssp, model_list):
    for model in model_list:
        print(model)
        df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
        huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
        huss = huss.sel(time = slice('2070-01-01T12:00:00', '2099-12-30T12:00:00'))

        df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
        Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
        Ta = Ta.sel(time = slice('2070-01-01T12:00:00', '2099-12-30T12:00:00')) - 273.15

        RH = (huss.huss * 1013 / (0.378 * huss.huss + 0.622)) / (6.112 * np.exp((17.67 * Ta.tas)/(Ta.tas + 243.5)))
        RH = xr.where(RH < 1, RH, 1)

        thi = (1.8 * Ta.tas + 32) - ( (0.55 - 0.55 * RH) * (1.8 * Ta.tas - 26) )
        thi = thi.chunk({'time': -1})
        thi = thi.groupby('time.dayofyear').median('time').to_dataset( name = 'THI' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/{ssp}/{model}_{ssp}_2070_2099_thi_doy_median.nc')




In [ ]:
run_ssp( 'ssp370', ssp370_models )

In [ ]:
run_ssp( 'ssp126', ssp126_models )
run_ssp( 'ssp245', ssp245_models )
run_ssp( 'ssp585', ssp585_models )

# Process model changes 

In [ ]:
histroot   = '/content/drive/My Drive/data/livestock/CMIP6/historical/'
ssp126root = '/content/drive/My Drive/data/livestock/CMIP6/ssp126/'
ssp585root = '/content/drive/My Drive/data/livestock/CMIP6/ssp585/'

hist   = os.listdir( histroot   )
ssp126 = os.listdir( ssp126root )
ssp585 = os.listdir( ssp585root )

ssp126 = fnmatch.filter( ssp126, '*doy*' )
for file in ssp126:
    model = file.split('_')[0]
    print( model )
    h    = xr.open_dataarray( histroot   + fnmatch.filter( hist,   f'{model}*doy*' )[0] )
    ssp1 = xr.open_dataarray( ssp126root + fnmatch.filter( ssp126, f'{model}*doy*' )[0] ) - h
    ssp5 = xr.open_dataarray( ssp585root + fnmatch.filter( ssp585, f'{model}*doy*' )[0] ) - h 
    ssp1.to_netcdf( f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp126_change_doy_median.nc4' )
    ssp5.to_netcdf( f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp585_change_doy_median.nc4' )

# resize CMIP6 and map change onto ERA baseline

In [ ]:
era = xr.open_dataarray('/content/drive/My Drive/data/livestock/ERA5/era_doy_media.nc4')
lon = era.lon.values
lat = era.lat.values
doy = era.dayofyear.values
pro_thresh = 66.5    # production threshold
era = era.values
era_thresh = np.ma.masked_where( era< pro_thresh, era )
era_pro = np.ma.count( era_thresh, axis = 0 )

In [ ]:
root = '/content/drive/My Drive/data/livestock/CMIP6/'
filelist = [root + i for i in fnmatch.filter( os.listdir(root), '*ssp*change*.nc4')]

for file in filelist:
    thiout = np.ma.zeros( ( 366, lat.shape[0], lon.shape[0] ) )
    out    = np.ma.zeros( ( 4, lat.shape[0], lon.shape[0] ) )
    model  = file.split( '/' )[-1].split( '_chang' )[0]
    print(model)
    thi = xr.open_dataarray( file )
    days = thi.dayofyear.values
    thi = thi.values

    if days.shape[0] == 366:
        for i in range(366): thiout[i,:,:] = np.flip(cv2.resize(thi[i,:,:], (1440, 721) ), axis = 0)

    elif days.shape[0] == 365:
        for i in range(60): thiout[i,:,:] = np.flip(cv2.resize(thi[i,:,:], (1440, 721) ), axis = 0)
        thiout[60,:,:] = np.flip(cv2.resize(thi[59,:,:], (1440, 721) ), axis = 0)
        for i in range(61,365): thiout[i,:,:] = np.flip(cv2.resize(thi[i-1,:,:], (1440, 721) ), axis = 0)

    elif days.shape[0] == 360:
        for i in range(0,30): thiout[i,:,:] = np.flip(cv2.resize(thi[i,:,:], (1440, 721) ), axis = 0)
        thiout[30,:,:] = np.flip(cv2.resize(thi[29,:,:], (1440, 721) ), axis = 0)
        for i in range(31,153): thiout[i,:,:] = np.flip(cv2.resize(thi[i-1,:,:], (1440, 721) ), axis = 0)
        thiout[153,:,:] = np.flip(cv2.resize(thi[152,:,:], (1440, 721) ), axis = 0)
        for i in range(154,214): thiout[i,:,:] = np.flip(cv2.resize(thi[i-3,:,:], (1440, 721) ), axis = 0)
        thiout[214,:,:] = np.flip(cv2.resize(thi[210,:,:], (1440, 721) ), axis = 0)
        for i in range(215,245): thiout[i,:,:] = np.flip(cv2.resize(thi[i-4,:,:], (1440, 721) ), axis = 0)
        thiout[245,:,:] = np.flip(cv2.resize(thi[240,:,:], (1440, 721) ), axis = 0)
        for i in range(246,306): thiout[i,:,:] = np.flip(cv2.resize(thi[i-5,:,:], (1440, 721) ), axis = 0)
        thiout[306,:,:] = np.flip(cv2.resize(thi[300,:,:], (1440, 721) ), axis = 0)    
        for i in range(306,365): thiout[i,:,:] = np.flip(cv2.resize(thi[i-6,:,:], (1440, 721) ), axis = 0)
        thiout[365,:,:] = np.flip(cv2.resize(thi[359,:,:], (1440, 721) ), axis = 0) 
    
    del thi    
    thiout = thiout + era

    ds = xr.Dataset( coords={'lat': lat, 'lon':  lon} )
    
    temp = np.ma.masked_where( thiout < pro_thresh, thiout ).count( axis = 0 )
    ds['pro_days'] = (['lat', 'lon'],  temp)
    temp = np.sum( np.ma.masked_where( thiout < pro_thresh, thiout ) - pro_thresh, axis = 0 )
    ds['pro_HL'] = (['lat', 'lon'],  temp)

    ds.to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_665thresh.nc4')

# OLD

---



---



---



In [ ]:
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv' )
 
#bad time 'SAM0-UNICON'
#no tas: 'FGOALS-f3-L'
# bad lat or lon ['EC-Earth3', 'EC-Earth3-Veg', 'CESM2-WACCM']
# too large 'CNRM-CM6-1-HR'

hist_models = ['NorESM2-LM','MPI-ESM-1-2-HAM','ACCESS-ESM1-5','MPI-ESM1-2-HR','MPI-ESM1-2-LR','BCC-CSM2-MR','KACE-1-0-G','UKESM1-0-LL','HadGEM3-GC31-LL']

for model in hist_models:
    print(model)
    df1 = df.query( f"activity_id=='CMIP' & variable_id == 'huss' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
    huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
    huss = huss.sel(time = slice('1985-01-01T12:00:00', '2014-12-30T12:00:00'))

    df1 = df.query( f"activity_id=='CMIP' & variable_id == 'tasmax' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
    Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
    Ta = Ta.sel(time = slice('1985-01-01T12:00:00', '2014-12-30T12:00:00')) - 273.15

    RH = (huss.huss * 1013 / (0.378 * huss.huss + 0.622)) / (6.112 * np.exp((17.67 * Ta.tasmax)/(Ta.tasmax + 243.5)))
    RH = xr.where(RH < 1, RH, 1)
    Tdp = 243.5 * (np.log(RH) + ( ( 17.67 * Ta.tasmax ) / ( 243.5 + Ta.tasmax ) ) ) / ( 17.67 - np.log(RH) - ( ( 17.67 * Ta.tasmax ) / ( 243.5 + Ta.tasmax ) ) )

    thi = Ta.tasmax + 0.36*Tdp + 41.2
    thi = thi.chunk({'time': -1})
    thiq = thi.quantile( np.arange(0.01,1, 0.01), dim = 'time' )
    thiq.to_dataset( name = 'THI_quants' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/historical/kibler/max/{model}_hist_1985_2014_thi_quantiles.nc')
    thi.groupby('time.dayofyear').median('time').to_dataset( name = 'THI' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/historical/kibler/max/{model}_hist_1985_2014_thi_doy_median.nc')

    thi = 0.80*Ta.tasmax + Ta.tasmax*RH - 1.4*RH + 46.4
    thi = thi.chunk({'time': -1})
    thiq = thi.quantile( np.arange(0.01,1, 0.01), dim = 'time' )
    thiq.to_dataset( name = 'THI_quants' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/historical/thom/max/{model}_hist_1985_2014_thi_quantiles.nc')
    thi.groupby('time.dayofyear').median('time').to_dataset( name = 'THI' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/historical/thom/max/{model}_hist_1985_2014_thi_doy_median.nc')

    df1 = df.query( f"activity_id=='CMIP' & variable_id == 'tas' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
    Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
    Ta = Ta.sel(time = slice('1985-01-01T12:00:00', '2014-12-30T12:00:00')) - 273.15

    RH = (huss.huss * 1013 / (0.378 * huss.huss + 0.622)) / (6.112 * np.exp((17.67 * Ta.tas)/(Ta.tas + 243.5)))
    RH = xr.where(RH < 1, RH, 1)
    Tdp = 243.5 * (np.log(RH) + ( ( 17.67 * Ta.tas ) / ( 243.5 + Ta.tas ) ) ) / ( 17.67 - np.log(RH) - ( ( 17.67 * Ta.tas ) / ( 243.5 + Ta.tas ) ) )

    thi = Ta.tas + 0.36*Tdp + 41.2
    thi = thi.chunk({'time': -1})
    thiq = thi.quantile( np.arange(0.01,1, 0.01), dim = 'time' )
    thiq.to_dataset( name = 'THI_quants' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/historical/kibler/mean/{model}_hist_1985_2014_thi_quantiles.nc')
    thi.groupby('time.dayofyear').median('time').to_dataset( name = 'THI' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/historical/kibler/mean/{model}_hist_1985_2014_thi_doy_median.nc')

    thi = 0.80*Ta.tas + Ta.tas*RH - 1.4*RH + 46.4
    thi = thi.chunk({'time': -1})
    thiq = thi.quantile( np.arange(0.01,1, 0.01), dim = 'time' )
    thiq.to_dataset( name = 'THI_quants' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/historical/thom/mean/{model}_hist_1985_2014_thi_quantiles.nc')
    thi.groupby('time.dayofyear').median('time').to_dataset( name = 'THI' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/historical/thom/mean/{model}_hist_1985_2014_thi_doy_median.nc')



In [ ]:
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv' )
 
#bad time 'SAM0-UNICON'
#no tas: 'FGOALS-f3-L'
# bad lat or lon ['EC-Earth3', 'EC-Earth3-Veg', 'CESM2-WACCM']
# too large 'CNRM-CM6-1-HR'

hist_models = ['NorESM2-LM','MPI-ESM-1-2-HAM','ACCESS-ESM1-5','MPI-ESM1-2-HR','MPI-ESM1-2-LR','BCC-CSM2-MR','KACE-1-0-G','UKESM1-0-LL','HadGEM3-GC31-LL']

for model in hist_models:
    print(model)
    df1 = df.query( f"activity_id=='CMIP' & variable_id == 'huss' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
    huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
    huss = huss.sel(time = slice('1985-01-01T12:00:00', '2014-12-30T12:00:00'))


    df1 = df.query( f"activity_id=='CMIP' & variable_id == 'tas' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
    Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
    Ta = Ta.sel(time = slice('1985-01-01T12:00:00', '2014-12-30T12:00:00')) - 273.15

    RH = (huss.huss * 1013 / (0.378 * huss.huss + 0.622)) / (6.112 * np.exp((17.67 * Ta.tas)/(Ta.tas + 243.5)))
    RH = xr.where(RH < 1, RH, 1)
    Tdp = 243.5 * (np.log(RH) + ( ( 17.67 * Ta.tas ) / ( 243.5 + Ta.tas ) ) ) / ( 17.67 - np.log(RH) - ( ( 17.67 * Ta.tas ) / ( 243.5 + Ta.tas ) ) )

    thi = Ta.tas + 0.36*Tdp + 41.2
    thi = thi.chunk({'time': -1})
    thiq = thi.quantile( np.arange(0.01,1, 0.01), dim = 'time' )
    thiq.to_dataset( name = 'THI_quants' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/historical/kibler/mean/{model}_hist_1985_2014_thi_quantiles.nc')
    thi.groupby('time.dayofyear').median('time').to_dataset( name = 'THI' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/historical/kibler/mean/{model}_hist_1985_2014_thi_doy_median.nc')

    thi = 0.80*Ta.tas + Ta.tas*RH - 1.4*RH + 46.4
    thi = thi.chunk({'time': -1})
    thiq = thi.quantile( np.arange(0.01,1, 0.01), dim = 'time' )
    thiq.to_dataset( name = 'THI_quants' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/historical/thom/mean/{model}_hist_1985_2014_thi_quantiles.nc')
    thi.groupby('time.dayofyear').median('time').to_dataset( name = 'THI' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/historical/thom/mean/{model}_hist_1985_2014_thi_doy_median.nc')



In [ ]:
def run_ssp(ssp, model_list):
    for model in model_list:
        print(model)
        df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
        huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
        huss = huss.sel(time = slice('2070-01-01T12:00:00', '2099-12-30T12:00:00'))

        df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'tasmax' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
        Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
        Ta = Ta.sel(time = slice('2070-01-01T12:00:00', '2099-12-30T12:00:00')) - 273.15

        RH = (huss.huss * 1013 / (0.378 * huss.huss + 0.622)) / (6.112 * np.exp((17.67 * Ta.tasmax)/(Ta.tasmax + 243.5)))
        RH = xr.where(RH < 1, RH, 1)
        Tdp = 243.5 * (np.log(RH) + ( ( 17.67 * Ta.tasmax ) / ( 243.5 + Ta.tasmax ) ) ) / ( 17.67 - np.log(RH) - ( ( 17.67 * Ta.tasmax ) / ( 243.5 + Ta.tasmax ) ) )

        thi = Ta.tasmax + 0.36*Tdp + 41.2
        thi = thi.chunk({'time': -1})
        thiq = thi.quantile( np.arange(0.01,1, 0.01), dim = 'time' )
        thiq.to_dataset(name='THI_quants').to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/{ssp}/kibler/max/{model}_{ssp}_2070_2099_thi_quantiles.nc')
        thi.groupby('time.dayofyear').median('time').to_dataset( name = 'THI' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/{ssp}/kibler/max/{model}_{ssp}_2070_2099_thi_doy_median.nc')

        thi = 0.80*Ta.tasmax + Ta.tasmax*RH - 1.4*RH + 46.4
        thi = thi.chunk({'time': -1})
        thiq = thi.quantile( np.arange(0.01,1, 0.01), dim = 'time' )
        thiq.to_dataset(name='THI_quants').to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/{ssp}/thom/max/{model}_{ssp}_2070_2099_thi_quantiles.nc')
        thi.groupby('time.dayofyear').median('time').to_dataset( name = 'THI' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/{ssp}/thom/max/{model}_{ssp}_2070_2099_thi_doy_median.nc')

        df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
        Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
        Ta = Ta.sel(time = slice('2070-01-01T12:00:00', '2099-12-30T12:00:00')) - 273.15

        RH = (huss.huss * 1013 / (0.378 * huss.huss + 0.622)) / (6.112 * np.exp((17.67 * Ta.tas)/(Ta.tas + 243.5)))
        RH = xr.where(RH < 1, RH, 1)
        Tdp = 243.5 * (np.log(RH) + ( ( 17.67 * Ta.tas ) / ( 243.5 + Ta.tas ) ) ) / ( 17.67 - np.log(RH) - ( ( 17.67 * Ta.tas ) / ( 243.5 + Ta.tas ) ) )

        thi = Ta.tas + 0.36*Tdp + 41.2
        thi = thi.chunk({'time': -1})
        thiq = thi.quantile( np.arange(0.01,1, 0.01), dim = 'time' )
        thiq.to_dataset(name = 'THI_quants').to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/{ssp}/kibler/mean/{model}_{ssp}_2070_2099_thi_quantiles.nc')
        thi.groupby('time.dayofyear').median('time').to_dataset( name = 'THI' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/{ssp}/kibler/mean/{model}_{ssp}_2070_2099_thi_doy_median.nc')

        thi = 0.80*Ta.tas + Ta.tas*RH - 1.4*RH + 46.4
        thi = thi.chunk({'time': -1})
        thiq = thi.quantile( np.arange(0.01,1, 0.01), dim = 'time' )
        thiq.to_dataset(name = 'THI_quants').to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/{ssp}/thom/mean/{model}_{ssp}_2070_2099_thi_quantiles.nc')
        thi.groupby('time.dayofyear').median('time').to_dataset( name = 'THI' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/{ssp}/thom/mean/{model}_{ssp}_2070_2099_thi_doy_median.nc')

#models = ['CanESM5','BCC-CSM2-MR']

#run_ssp( 'ssp585', models )


In [ ]:
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv' )

def run_ssp(ssp, model_list):
    for model in model_list:
        print(model)
        df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
        huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
        huss = huss.sel(time = slice('2070-01-01T12:00:00', '2099-12-30T12:00:00'))

        df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
        Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
        Ta = Ta.sel(time = slice('2070-01-01T12:00:00', '2099-12-30T12:00:00')) - 273.15

        RH = (huss.huss * 1013 / (0.378 * huss.huss + 0.622)) / (6.112 * np.exp((17.67 * Ta.tas)/(Ta.tas + 243.5)))
        RH = xr.where(RH < 1, RH, 1)

        thi = 0.80*Ta.tas + Ta.tas*RH - 1.4*RH + 46.4
        thi = thi.chunk({'time': -1})
        thiq = thi.quantile( np.arange(0.01,1, 0.01), dim = 'time' )
        thiq.to_dataset(name = 'THI_quants').to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/{ssp}/thom/mean/{model}_{ssp}_2070_2099_thi_quantiles.nc')
        thi.groupby('time.dayofyear').median('time').to_dataset( name = 'THI' ).to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/{ssp}/thom/mean/{model}_{ssp}_2070_2099_thi_doy_median.nc')

run_ssp( 'ssp126',  ['BCC-CSM2-MR','CanESM5','MPI-ESM1-2-HR','KACE-1-0-G','UKESM1-0-LL','GFDL-ESM4','CNRM-CM6-1','CNRM-ESM2-1','MRI-ESM2-0'] )


In [ ]:
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv' )

In [ ]:
models = ['MPI-ESM1-2-HR',
 'CNRM-CM6-1-HR',
 'MRI-ESM2-0',
 'CNRM-CM6-1',
 'UKESM1-0-LL',
 'CanESM5',
 'KACE-1-0-G',
 'BCC-CSM2-MR',
 'GFDL-ESM4',
 'CNRM-ESM2-1']

run_ssp('ssp126', models)



---

# OLD

---



In [ ]:
years = np.arange(1985,2015,1)
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

for year in years:
    for month in months:
        print(year, month)
        client.download_file( era5_bucket, f'{year}/{month}/data/air_temperature_at_2_metres.nc', 
                            f'/content/{year}_{month}_air_temperature_at_2_metres.nc' )

        client.download_file( era5_bucket, f'{year}/{month}/data/dew_point_temperature_at_2_metres.nc', 
                            f'/content/{year}_{month}_dew_point_temperature_at_2_metres.nc' )

        Ta = xr.open_dataarray(f'/content/{year}_{month}_air_temperature_at_2_metres.nc') - 273.15
        Tdp = xr.open_dataarray(f'/content/{year}_{month}_dew_point_temperature_at_2_metres.nc') - 273.15
        RH  = np.exp( ( 17.625 * Tdp ) / ( 243.04 + Tdp ) ) / np.exp( ( 17.625 * Ta ) / ( 243.04 + Ta ) )

        thi = Ta + 0.36*Tdp + 41.2
        thim = thi.resample( time0 = '1D' ).max()
        thim.to_dataset(name='THI').to_netcdf(f'/content/drive/My Drive/data/livestock/ERA5/kibler/max/{year}_{month}_daily_max_thi.nc')
        thim = thi.resample( time0 = '1D' ).mean()
        thim.to_dataset(name='THI').to_netcdf(f'/content/drive/My Drive/data/livestock/ERA5/kibler/mean/{year}_{month}_daily_mean_thi.nc')
        
        del thi 
        
        RH  = np.exp( ( 17.625 * Tdp ) / ( 243.04 + Tdp ) ) / np.exp( ( 17.625 * Ta ) / ( 243.04 + Ta ) )
        del Tdp

        thi = 0.80*Ta + Ta*RH - 1.4*RH + 46.4
        thim = thi.resample( time0 = '1D' ).max()
        thim.to_dataset(name='THI').to_netcdf(f'/content/drive/My Drive/data/livestock/ERA5/thom/max/{year}_{month}_daily_max_thi.nc')

        thim = thi.resample( time0 = '1D' ).mean()
        thim.to_dataset(name='THI').to_netcdf(f'/content/drive/My Drive/data/livestock/ERA5/thom/mean/{year}_{month}_daily_mean_thi.nc')

        del thi

        os.remove(f'/content/{year}_{month}_air_temperature_at_2_metres.nc')
        os.remove(f'/content/{year}_{month}_dew_point_temperature_at_2_metres.nc')


In [ ]:
keys = []
date = datetime.date( 1985, 1, 1 ) # update to desired date
prefix = date.strftime( '%Y/%m/' )

response = client.list_objects_v2( Bucket=era5_bucket, Prefix=prefix )
response_meta = response.get( 'ResponseMetadata' )

if response_meta.get('HTTPStatusCode') == 200:
    contents = response.get('Contents')
    if contents == None: print("No objects are available for %s" % date.strftime('%B, %Y'))
    else:
        for obj in contents: keys.append(obj.get('Key'))
        print( "There are %s objects available for %s\n--" % (len(keys), date.strftime('%B, %Y')))
        for k in keys: print(k)
else: print( "There was an error with your request." )

In [ ]:
ds = xr.open_dataarray(f'/content/{year}_{month}_air_temperature_at_2_metres.nc')
ds.metpy.convert_units('degC')

dsd = xr.open_dataarray(f'/content/{year}_{month}_dew_point_temperature_at_2_metres.nc')
dsd.metpy.convert_units('degC')

rh = mpcalc.relative_humidity_from_dewpoint( ds , dsd ) 

In [ ]:
### Reference ###

In [ ]:
def THI(spec, Ta, Tdp = False, huss = False):
    # takes dry-bulb temperature in C and either dewpoint (C) or specific humidity (kg/kg)
    # returns THI based on whichever specificciation you choose
    Ta = np.nan_to_num( np.atleast_1d( Ta ) )
    temp  = Ta * units.degC
    press = np.array( 1e5 ) * units.hectopascal

    if huss == False: 
        Tdp = np.nan_to_num( np.atleast_1d( Tdp ) )
        dew = Tdp * units.degC 
        RH  = np.asarray( mpcalc.relative_humidity_from_dewpoint( temp , dew ) )
        ### Wet bulb from Stull, 2011 Journal of Applied Meterology and Climatology, Equation 1
        #Twb = Ta * np.arctan( 0.151977 * (RH + 8.313659)**(1/2) ) + np.arctan(Ta+ RH) - np.arctan(RH - 1.676331) + 0.00391838 * (RH)**(3/2) * np.arctan(0.023101 * RH) - 4.686035

    elif Tdp == False:
        huss = np.nan_to_num( np.atleast_1d( huss ) )
        SH  =  huss * units.dimensionless
        RH  = np.asarray( mpcalc.relative_humidity_from_specific_humidity(SH, temp, press ) )
        Tdp = np.asarray( mpcalc.dewpoint_from_specific_humidity( SH, temp, press ) )

    if   spec == 'thom':   thi = 0.80*Ta + Ta*RH - 1.4*RH + 46.4      ## adapted from Thom59, they do something goofy with the conversion
    elif spec == 'kibler': thi = 1.00*Ta + 0.36*Tdp + 41.2            ## Kibler 64, NRC71, then yousef85

    thi = np.ma.masked_outside( np.nan_to_num( thi ), 1, 200 ).compressed()

    return thi

In [ ]:
ds.temperature.metpy.convert_units('degC')
ds.

thiT = 0.80*Ta + Ta*RH - 1.4*RH + 46.4      
thiK = 1.00*Ta + 0.36*Tdp + 41.2            


In [ ]:
!pip install --upgrade botocore s3fs

In [ ]:
import datetime
from dask.distributed import LocalCluster, Client
import s3fs
import boto3

In [ ]:
import botocore

In [ ]:
era5_bucket = 'era5-pds'
client = boto3.client('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED))

In [ ]:
keys = []
date = datetime.date( 2019, 1, 1 ) # update to desired date
prefix = 'zarr/2008/01/data/'  

response = client.list_objects_v2( Bucket=era5_bucket, Prefix=prefix )
response_meta = response.get( 'ResponseMetadata' )

if response_meta.get('HTTPStatusCode') == 200:
    contents = response.get('Contents')
    if contents == None: print("No objects are available for %s" % date.strftime('%B, %Y'))
    else:
        for obj in contents: keys.append(obj.get('Key'))
        print( "There are %s objects available for %s\n--" % (len(keys), date.strftime('%B, %Y')))
        for k in keys: print(k)
else: print( "There was an error with your request." )

In [ ]:
fs = s3fs.S3FileSystem(anon=True)

In [ ]:
def inc_mon(indate):
    if indate.month < 12:
        return datetime.datetime(indate.year, indate.month+1, 1)
    else:
        return datetime.datetime(indate.year+1, 1, 1)

def gen_d_range(start, end):
    rr = []
    while start <= end:
        rr.append(start)
        start = inc_mon(start)
    return rr

def get_z(dtime,var):
    f_zarr = 'era5-pds/zarr/{year}/{month:02d}/data/{var}.zarr/'.format(year=dtime.year, month=dtime.month,var=var)
    return xr.open_zarr(s3fs.S3Map(f_zarr, s3=fs))

def gen_zarr_range(start, end,var):
    return [get_z(tt,var) for tt in gen_d_range(start, end)]

In [ ]:
tmp_a = gen_zarr_range(datetime.datetime(1980,1,1), datetime.datetime(1980,3,31),'air_temperature_at_2_metres')
tmp_all = xr.concat(tmp_a, dim='time0')
tmp_all = tmp_all.air_temperature_at_2_metres.chunk({'time0': -1}).resample(time0='1D').max()
tmp_all = tmp_all.quantile( [0.9,0.95], dim = 'time0' )
tmp_all.to_netcdf('/content/drive/My Drive/data/livestock/t2m_90_quantile.nc4')

In [ ]:
resample(time0='1D').quantile( [0.9,0.95], dim = 'time0' )

In [ ]:
tmp_all

# Climate data

In [ ]:
ds = xr.open_zarr( fsspec.get_mapper('gcs://pangeo-era5/reanalysis/temporal-analysis'), 
                  consolidated = True, chunks = 'auto')# {'time': -1, 'latitude':10, 'longitude':10}  )
ds

In [ ]:
path = '/content/drive/My Drive/data/livestock/t2m_90_quantile.nc4'
quants = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99]
ds.t2m.quantile( quants, dim = 'time' ).to_netcdf(path)

In [ ]:
path = '/content/drive/My Drive/data/livestock/1984_d2m_90up_quantile.nc4'
quants = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99]
with ProgressBar():
    ds.d2m.sel( time = slice('1984-01-01T00:00:00', '1984-01-01T23:00:00') ).to_netcdf(path)

In [ ]:
d = ds.t2m.sel( time = '1984-01-01T00:00:00' ).values

In [ ]:
ds.t2m.sel(time='2000-12-31T23:00:00').squeeze().plot()

In [ ]:
d = d.to_netcdf(path)

In [ ]:
path = '/content/drive/My Drive/data/livestock/1984_d2m_90up_quantile.nc4'
quants = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99]
d = ds.t2m.sel(time = slice('1984-01-01T00:00:00', '1984-01-31T23:00:00')).quantile( quants, dim = 'latitude' )

with ProgressBar():
    out = d.compute()

In [ ]:
path = '/content/drive/My Drive/data/livestock/t2m_90_quantile.nc4'
quants = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99]
ds.t2m.sel( latitude = slice(90,87.5) ).resample( time = '1D' ).max().quantile( quants, dim = 'time' ).to_netcdf(path)

### CMIP6

In [ ]:
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv' )
df = df.query( "activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp585' & source_id == 'IPSL-CM6A-LR' & table_id == 'day'" )
df

In [ ]:
ds = xr.open_zarr( gcs.get_mapper( df.zstore.values[0] ), consolidated = True )

In [ ]:
import time
import xarray as xr
import numpy as np

from tqdm import tqdm_notebook
from xarray.core.groupby import DataArrayGroupBy, DatasetGroupBy

def inner_generator(df_function='apply'):
    def inner(df, func, *args, **kwargs):
        
        t = tqdm_notebook(total = len(df))
        def wrapper(*args, **kwargs):
            t.update( n=1 if not t.total or t.n < t.total else 0 )
            return func( *args, **kwargs )

        result = getattr(df, df_function)(wrapper, **kwargs)

        t.close()
        return result
    return inner

DataArrayGroupBy.progress_apply = inner_generator()
DatasetGroupBy.progress_apply = inner_generator()

arr = xr.DataArray( np.arange( 10 ), dims = [ 'x' ] )

def foo( v ):
    time.sleep( 1 )
    return v + 15

print( arr.groupby( 'x' ).progress_apply( foo ) )